In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import re
import sys 
import statsmodels.stats.multitest
import math as math
import scipy.stats
import re
import sys
import cptac
import cptac.utils as u
import plot_utils as p 

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:

import statsmodels.stats.multitest
from bokeh.palettes import RdBu
from bokeh.models import LinearColorMapper, ColumnDataSource, ColorBar
from bokeh.models.ranges import FactorRange
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.io import export_png
from bokeh.io import export_svgs
from bokeh.layouts import row

from bokeh.plotting import figure, output_file, show
from bokeh.models import LogColorMapper, LogTicker, ColorBar



def plotCircleHeatMap ( df, circle_var, color_var, x_axis, y_axis,plot_width= 1000, plot_height = 650, x_axis_lab = "no_label", y_axis_lab = "no_label", show_plot = True, save_png = "plot.png"):
  
    # circle_var designed for pvalues. Normalized by taking log 10 of values and multiplying by 5 
    #added a new column to make the plot size
    
    df["size2"] = df[circle_var].apply(lambda x: -1*(np.log(x)))
    df['size'] = (df["size2"])*3
    #find values to set color bar min/ max as 
    maxval = df[color_var].max()
    minval = df[color_var].min()
    if maxval > abs(minval):
        minval = maxval * -1 
    if maxval < abs(minval):
        maxval = minval * -1
    colors = list((RdBu[9]))
    exp_cmap = LinearColorMapper(palette=colors, low = minval, high = maxval)
    p = figure(x_range = FactorRange(), y_range = FactorRange(), plot_width= plot_width, 
               plot_height=plot_height, 
               toolbar_location=None, tools="hover")

    p.scatter(x_axis,y_axis,source=df, fill_alpha=1,  line_width=0, size="size", 
              fill_color={"field":color_var, "transform":exp_cmap})

    p.x_range.factors = sorted(df[x_axis].unique().tolist())
    p.y_range.factors = sorted(df[y_axis].unique().tolist(), reverse = True)
    p.xaxis.major_label_orientation = math.pi/2
    
    if (x_axis_lab != "no_label" ):
        p.xaxis.axis_label = x_axis_lab
    if (x_axis_lab != "no_label" ):   
        p.yaxis.axis_label = y_axis_lab

    bar = ColorBar(color_mapper=exp_cmap, location=(0,0))
    p.add_layout(bar, "right")
    color_bar_height = plot_height + 10
    color_bar_width = 35
    
    color_bar_plot = figure(title="My color bar title", title_location="right", 
                        height=color_bar_height, width=color_bar_width, 
                        toolbar_location=None, min_border=0, 
                        outline_line_color=None)
   # color_bar_plot.title.text_font_size = '12pt'
    color_bar_plot.title.align="center"
 
    if show_plot:
        output_notebook()
        layout = row(p, color_bar_plot)
        show(layout)
      
    if save_png != "plot.png":
        export_png(p, filename= save_png)

In [23]:


test_genes = ["VEGFA","VEGFC","ANGPTL4","ANGPTL2","TGFB1","TGFBR1",'TGFB2','TGFBR3',"ANG",'TNFAIP2',
 'TNFAIP3',
 'TNFAIP6',
 'TNFAIP8',
 'TNFAIP8L2',
 'TNFAIP8L3',
 'TNFRSF12A',
 'TNFRSF17',
 'TNFRSF1A',
 'TNFSF10',
"TNF","CTSB","ITGAV","ITGB3","AGTRAP"]

#genes related to angiogenisis 
test_genes = ["AGTRAP","ANGPTL4","TGFB1","TGFB2","ITGAV","ITGB3","CTSB","MMP9","MMP3","MMP2","PLAU","PDL1"]

test_genes = ["PHLDA1","PHLDA2","PHLDA3"]
test_genes = ["CD55","CD59","CD46","CFH","CFI","PRKCZ","FOXP3","C5AR1","C3AR1"]
prot_column_names = []
for gene in test_genes:
    gene += "_proteomics"
    prot_column_names.append(gene)

prot_column_names

['CD55_proteomics',
 'CD59_proteomics',
 'CD46_proteomics',
 'CFH_proteomics',
 'CFI_proteomics',
 'PRKCZ_proteomics',
 'FOXP3_proteomics',
 'C5AR1_proteomics',
 'C3AR1_proteomics']

In [24]:
df_FDR_append = pd.read_csv("../Step3.2_combining_pearson_dfs/csv_files/pancan_EGFR_pearson_sig_all_prot_append_FDR.csv")
df_FDR_append = df_FDR_append.drop(['Unnamed: 0'], axis=1)

df_FDR_test = df_FDR_append[df_FDR_append.Comparison.isin(prot_column_names)]
df_FDR_test

,Comparison,Correlation,P_value,fdr_bh_p_val,Cancer Type
402,C3AR1_proteomics,-0.387193,0.000075,0.001997,GBM
556,CFI_proteomics,-0.363898,0.000213,0.004091,GBM
641,C5AR1_proteomics,-0.353655,0.000330,0.005494,GBM
2869,CFH_proteomics,0.359191,0.000854,0.045026,Ovarian
4489,PRKCZ_proteomics,-0.290437,0.001640,0.010407,Brca
4670,CFH_proteomics,0.279777,0.002460,0.014016,Brca
6467,CFH_proteomics,-0.292300,0.001944,0.026045,Luad
6640,CD59_proteomics,0.278881,0.003175,0.034929,Luad


In [5]:
plotCircleHeatMap(df_FDR_test, "P_value","Correlation","Comparison","Cancer Type",plot_width= 800, plot_height = 850)

Loading BokehJS ...

ValueError: Out of range float values are not JSON compliant

In [15]:
df = pd.read_csv("../Step3.3_Pos_Neg_Correlation_patterns/csv_files/pancan_EGFR_Gbm_pos.csv")

df = df.set_index('Comparison')
df1_transposed = df.T 
df1_transposed



all_prot = df1_transposed.columns.values.tolist()


In [16]:
df_FDR_append = pd.read_csv("../Step3.2_combining_pearson_dfs/csv_files/pancan_EGFR_pearson_sig_all_prot_append_FDR.csv")
df_FDR_append = df_FDR_append.drop(['Unnamed: 0'], axis=1)

df_FDR_test = df_FDR_append[df_FDR_append.Comparison.isin(all_prot)]
df_FDR_test

,Comparison,Correlation,P_value,Cancer Type
1,PHLDA1_proteomics,0.816848,6.553435e-25,GBM
3,SOCS2_proteomics,0.562720,1.343464e-09,GBM
15,PHLDA3_proteomics,0.525883,2.274919e-08,GBM
34,EPHB3_proteomics,0.496019,1.782066e-07,GBM
38,CLIP2_proteomics,0.493280,2.131667e-07,GBM
...,...,...,...,...
8658,GPRC5C_proteomics,0.325740,6.967303e-04,Lscc
8688,TANC2_proteomics,0.312878,8.748102e-04,Lscc
8747,PTPRZ1_proteomics,0.301326,1.379432e-03,Lscc
8803,MYO10_proteomics,0.292588,1.923428e-03,Lscc


In [13]:
p.plotCircleHeatMap(df_FDR_test, "P_value","Correlation","Comparison","Cancer Type",plot_width= 1500, plot_height = 1550)

Loading BokehJS ...

In [ ]:
#LANCL, PHLDA1,2,3